Лабораторна робота № 5
Смислов Даніл
ІП-01

Спочатку імпортуємо потрібні для роботи бібліотеки

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np


Далі імпортуємо дані з файлу wine.csv з кодуванням 'cp1252' та розділювачем ','. Розглянемо структуру датафрейму.

In [ ]:
def getDataset():
    data = pd.read_csv('wine.csv',sep=',',encoding='cp1252')
    return data 

dataset = getDataset()
dataset.info()
dataset.head(10)

Як ми бачимо, з типом проблем немає. Давайте перевіримо на негативні значення.

In [ ]:
for column in dataset.columns:
    print(column + ": ")
    print(dataset[dataset[column]<0])

Як ми бачимо, негативних значень немає. Тому можемо працювати з даними. Давайте перевіримо наші значення в колонках на нормальність розподілу, використовуючи критерій Пірсона, адже розмір вибірки є досить великим.


In [ ]:
import scipy.stats as stats
def testPirsona(data):
    statistic, p = stats.normaltest(data)
    print('Pirsona:', 'Statistic: %.4f P: %.30f' % (statistic,p))
    if(p > 0.05):
        print('Нормальний розподіл')
    else:
        print(len(data))
        print('Не нормальний розподіл')

for column in dataset.columns:
    print(column)
    testPirsona(dataset[column])


Як бачимо, розподіли не нормального виду. Давайте розрахуємо матрицю кореляції для даних.

In [ ]:
dataset.corr()

Давайте побудуємо діаграму розмаху для стовпчика 'chlorides', до прикладу.

In [ ]:
 
dataset.boxplot('chlorides')


Як ми бачимо, є "викиди" та значення досить сильно різняться. Давайте напишемо функцію для відкидання "викидів" та застосуємо до стовпця. 

In [ ]:
def deleteEmission(dataset,columnName):
    Q1 = dataset[columnName].quantile(.25)
    Q3 = dataset[columnName].quantile(.75)
    center = Q3 - Q1
    dataset = dataset.drop(dataset[dataset[columnName]<(Q1-1.5* center)].index)
    dataset = dataset.drop(dataset[dataset[columnName]>(Q3 + 1.5* center)].index)
    return dataset

dataset = deleteEmission(dataset,'chlorides')
dataset.boxplot('chlorides')


Як ми бачимо, тепер немає таких виражених "викидів". Тепер застосуємо нашу функцію до всіх стовпців.

In [ ]:
for columnName in dataset.columns:
    dataset = deleteEmission(dataset,columnName)

dataset.boxplot('fixed acidity')

Тепер давайте виокремимо дані та розподілимо їх на навчальну та тестові вибірки.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
data = dataset.iloc[:,:11].to_numpy()
fact = dataset['quality'].to_numpy()

trainingX,testX,trainingY,testY = train_test_split(data,fact,test_size = 0.33,random_state = 0)



Тепер побудуємо лінійну та поліноміальні моделі 2-3 порядків для прогнозу 'quality'.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
linearRegression = LinearRegression().fit(trainingX,trainingY)
models = []
models.append(linearRegression)
for i in range(2,4):
    print(i)
    tempPipeline = make_pipeline(PolynomialFeatures(degree=i), LinearRegression())
    tempReg = tempPipeline.fit(trainingX,trainingY)
    models.append(tempReg)

Тепер давайте обрахуємо MSE та R2

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
MSE = []
R2 = []
for model in models:
    tempPred = model.predict(testX)
    MSE.append(mean_squared_error(testY,tempPred))
    R2.append(r2_score(testY,tempPred))

In [ ]:
degrees = [1,2,3]
plt.plot(degrees,MSE,color = 'green')
plt.plot(degrees,R2,color = 'red')
plt.show()
print(R2)

Як ми бачимо, моделі 1 та 2 порядків досить схожі між собою, модель 3 порядку показує себе вже гірше. Тому найкращою буде модель 1 порядку, адже вона простіша за модель 2 порядку. Тепер приступимо до виконання додаткового завдання. Спочатку зчитаємо датасет.

In [ ]:
def getDataset():
    data = pd.read_csv('Data4.csv',sep=';',encoding='windows-1251',decimal = ',')
    return data 

dataset = getDataset()
dataset.info()
dataset.head(10)

З типами проблем немає. Давайте перейменуємо першу колонку.

In [ ]:
dataset.rename(columns = {'Unnamed: 0':'Country'})

Давайте перевіримо на негативні значення.

In [ ]:
for column in dataset.columns:
    if dataset[column].dtype != 'object':
        print(column + ": ")
        print(dataset[dataset[column]<0])

Як бачимо, негативних значень немає. Ми готові працювати з цими даними. Давайте розрахуємо матрицю кореляції для даних. 


In [ ]:
dataset.corr()

Тепер давайте побудуємо діграми розсіювання.

In [ ]:
pd.plotting.scatter_matrix(dataset,figsize=(12,12))
plt.show()

Давайте побудуємо регресійні моделі. Спочатку лінійні для всіх наборів предикторів.

In [ ]:
Y = dataset['Cql']

linear = []

linear.append(LinearRegression().fit(dataset['Ie'].to_numpy().reshape(-1,1), Y))
linear.append(LinearRegression().fit(dataset['Iec'].to_numpy().reshape(-1,1), Y))
linear.append(LinearRegression().fit(dataset['Is'].to_numpy().reshape(-1,1), Y))
linear.append(LinearRegression().fit(dataset[['Is', 'Iec']], Y))
linear.append(LinearRegression().fit(dataset[['Is', 'Ie']], Y))
linear.append(LinearRegression().fit(dataset[['Ie', 'Iec']], Y))
linear.append(LinearRegression().fit(dataset[['Ie', 'Iec','Is']], Y))

Тепер давайте створимо поліноміальні моделі 2 порядку

In [ ]:
polinomial = []
polinomial.append(make_pipeline(PolynomialFeatures(degree=2),LinearRegression()))
polinomial[0].fit(dataset['Ie'].to_numpy().reshape(-1,1), Y)
polinomial.append(make_pipeline(PolynomialFeatures(degree=2),LinearRegression()))
polinomial[1].fit(dataset['Iec'].to_numpy().reshape(-1,1), Y)
polinomial.append(make_pipeline(PolynomialFeatures(degree=2),LinearRegression()))
polinomial[2].fit(dataset['Is'].to_numpy().reshape(-1,1), Y)
polinomial.append(make_pipeline(PolynomialFeatures(degree=2),LinearRegression()))
polinomial[3].fit(dataset[['Is', 'Iec']], Y)
polinomial.append(make_pipeline(PolynomialFeatures(degree=2),LinearRegression()))
polinomial[4].fit(dataset[['Is', 'Ie']], Y)
polinomial.append(make_pipeline(PolynomialFeatures(degree=2),LinearRegression()))
polinomial[5].fit(dataset[['Ie', 'Iec']], Y)
polinomial.append(make_pipeline(PolynomialFeatures(degree=2),LinearRegression()))
polinomial[6].fit(dataset[['Ie', 'Iec','Is']], Y)

Тепер давайте зчитаємо датасет з даними для тесту та протестуємо наші моделі.

In [ ]:
def getDataset():
    data = pd.read_csv('Data4.csv',sep=';',encoding='windows-1251',decimal = ',')
    return data 

testData = getDataset()
testData.info()
testData.head(10)


З типами проблем немає. Давайте перейменуємо колонку. 

In [ ]:
testData.rename(columns = {'Unnamed: 0':'Country'})

Тепер готові працювати з даними. Давайте порахуємо передбачувані дані для лінійних моделей, які ми побудували.

In [ ]:
predicts = []

predicts.append(linear[0].predict(testData['Ie'].to_numpy().reshape(-1,1)))
predicts.append(linear[1].predict(testData['Iec'].to_numpy().reshape(-1,1)))
predicts.append(linear[2].predict(testData['Is'].to_numpy().reshape(-1,1)))
predicts.append(linear[3].predict(testData[['Is','Iec']]))
predicts.append(linear[4].predict(testData[['Is', 'Ie']]))
predicts.append(linear[5].predict(testData[['Ie', 'Iec']]))
predicts.append(linear[6].predict(testData[['Ie', 'Iec','Is']]))

Тепер також додамо для поліноміальних.

In [ ]:
predicts.append(polinomial[0].predict(testData['Ie'].to_numpy().reshape(-1,1)))
predicts.append(polinomial[1].predict(testData['Iec'].to_numpy().reshape(-1,1)))
predicts.append(polinomial[2].predict(testData['Is'].to_numpy().reshape(-1,1)))
predicts.append(polinomial[3].predict(testData[['Is','Iec']]))
predicts.append(polinomial[4].predict(testData[['Is', 'Ie']]))
predicts.append(polinomial[5].predict(testData[['Ie', 'Iec']]))
predicts.append(polinomial[6].predict(testData[['Ie', 'Iec','Is']]))

Тепер звіримо передбачувані значення з реальними значеннями в датасеті та визначимо яка з моделей краща.

In [ ]:
indexMinMSE = np.sum((np.array(predicts)- testData['Cql'].to_numpy())**2,axis = 1).argmin()
print(indexMinMSE)

Отже, найменше значення MSE показує модель з індексом 13, тобто поліноміальна з 3 змінними, що доволі логічно.